In [5]:
import tensorflow

In [29]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import datetime

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [12]:
data = pd.read_csv("../data/MSFT.csv")
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index("Date")
data = data.diff().dropna()

In [ ]:
# last_date = data.index[-1]

# # Create a 2-week offset
# offset = timedelta(weeks=2)

# start = '2009-05-08'
# end = last_date-offset

# dataframe = data.copy()
# dataframe = dataframe.loc[start:end, :]
# dataframe

In [42]:
def load_data(start, end=None):
    last_date = data.index[-1]
    offset = timedelta(weeks=2)

    dataframe = data.copy()
    if not end:
        end = last_date-offset

    dataframe = dataframe.loc[start:end, :]
    return dataframe.rename(columns = {'Closing_Price': 'Close'})

In [43]:
train_set = load_data('2009-05-08')

In [44]:
# Normalize data
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(train_set['Close'].values.reshape(-1,1))

# how many days do i want to base my predictions on ?
prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x - prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [45]:
def LSTM_model():
    
    model = Sequential()
    
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1],1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=1))
    
    return model

In [46]:
from keras.callbacks import EarlyStopping
from keras.metrics import RootMeanSquaredError

model = LSTM_model()
model.summary()
model.compile(optimizer='adam', 
              loss='mean_squared_error')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 60, 50)            10400     
                                                                 
 dropout_3 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_4 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_4 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                

In [47]:
checkpointer = ModelCheckpoint(filepath = '../model/weights_best.hdf5', 
                               verbose = 1, 
                               save_best_only = True)


history = model.fit(x_train, 
          y_train, 
          validation_split=0.2,
          epochs=25, 
          batch_size = 32,
          callbacks = [checkpointer])

Epoch 1/25
85/85 [==============================] - ETA: 0s - loss: 0.0191
Epoch 1: val_loss improved from inf to 0.01205, saving model to ../model\weights_best.hdf5
85/85 [==============================] - 9s 50ms/step - loss: 0.0191 - val_loss: 0.0121
Epoch 2/25
85/85 [==============================] - ETA: 0s - loss: 0.0036
Epoch 2: val_loss improved from 0.01205 to 0.01192, saving model to ../model\weights_best.hdf5
85/85 [==============================] - 3s 34ms/step - loss: 0.0036 - val_loss: 0.0119
Epoch 3/25
85/85 [==============================] - ETA: 0s - loss: 0.0031
Epoch 3: val_loss did not improve from 0.01192
85/85 [==============================] - 3s 33ms/step - loss: 0.0031 - val_loss: 0.0120
Epoch 4/25
85/85 [==============================] - ETA: 0s - loss: 0.0029
Epoch 4: val_loss did not improve from 0.01192
85/85 [==============================] - 3s 33ms/step - loss: 0.0029 - val_loss: 0.0121
Epoch 5/25
85/85 [==============================] - ETA: 0s - loss: 

In [48]:
last_date = data.index[-1]
offset = timedelta(weeks=2)

test_data = load_data(start = last_date - offset,
                      end = last_date)

actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)

x_test = [
    model_inputs[x - prediction_days : x, 0]
    for x in range(prediction_days, len(model_inputs))
]
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1] ,1))

predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

1/1 [==============================] - 1s 1s/step


In [49]:
predicted_prices

array([[-0.31246108],
       [-0.28739554],
       [-0.26632956],
       [-0.25694317],
       [-0.26507357],
       [-0.2821271 ],
       [-0.29732755],
       [-0.3063639 ],
       [-0.3045134 ],
       [-0.29149806]], dtype=float32)

In [23]:
# validade model

test_loss, test_rmse = model.evaluate(x_test, y_test)

NameError: name 'x_test' is not defined

In [ ]:
# test_model

import numpy as np
#features = [Open, High, Low]
features = np.array([[177.089996, 180.419998, 177.070007]])
model.predict(features)

In [ ]:
# plot test and prediction

import numpy as np
import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure([
    go.Scatter(x=X_train.squeeze(), y=y_train, name='train', mode='markers'),
    go.Scatter(x=X_test.squeeze(), y=y_test, name='test', mode='markers'),
    go.Scatter(x=x_range, y=y_range, name='prediction')
])
fig.show()